# Concordancer. (outputs raw and xml)

In [21]:
import re

In [29]:
input_file = '''ཉེ་ཉོན་ཉི་ཤུ་ལས། 
ཁྲོ་བ་ནི་ཁོང་ཁྲོ་འཕེལ་ཏེ་བརྡེག་པ་སོགས་གནོད་པ་དངོས་སུ་ཤོམ་པར་བྱེད་པའོ། །
འཁོན་དུ་འཛིན་པ་ནི་ཁོང་ཁྲོ་བའི་ཆར་གཏོགས་པ་གནོད་པའི་བསམ་པ་རྒྱུན་མི་གཏོང་ཞིང་མི་བཟོད་པར་བྱེད་པའོ། །
འཚིག་པ་ནི་ཁྲོ་བ་དང་འཁོན་འཛིན་གྱི་རྒྱུ་ལས་མི་བཟོད་པར་ཚིག་རྩུབ་སྨྲ་བར་བྱེད་པའོ། །
རྣམ་པར་འཚེ་བ་ནི་ཁོང་ཁྲོའི་ཆར་གཏོགས་པ། སྙིང་བརྩེ་བ་མེད་པར་རྣམ་པར་མཐོ་བཙམ་པའི་ལས་ཅན་ནོ། །'''

plain_conc = []
xml_entries = []
# lines : cut the file in units separated by spaces
mot = "འདུལ་བ་"
left = 5
right = 5

occur_num = 1
for line_number, line in enumerate(input_file):
    if mot in line:
        # find the positions of the occurences
        syllabled = joinedword_list(mot, line)
        positions = []
        for syl_num, syl in enumerate(syllabled):
            stripped_syl = syl.strip("།")
            if mot == stripped_syl:
                positions.append(syl_num)
                
        # establishes right and left context
        for pos in positions:
            middle = syllabled[pos]
            
            context_left = ""
            if len(syllabled[0:pos]) <= left:
                if len(syllabled[0:pos]) == 0:
                    context_left = ""
                else:
                    context_left = "".join(syllabled[0:pos])
            else:
                context_left = "".join(syllabled[pos-9:pos])
                
            context_right = ""
            if len(syllabled[pos+1:]) <= right:
                if len(syllabled[pos+1:]) == 0:
                    context_right = ""
                else:
                    context_right = "".join(syllabled[pos+1:])
            else:
                context_right = "".join(syllabled[pos+1:pos+1+right])
            
            plain_conc.append(str(line_number+1)+"\t"+context_left+" -"+str(occur_num)+"|"+middle+"|- "+context_right)
            #print(str(line_number+1)+"\t"+context_left+" -"+str(occur_num)+"|"+middle+"|- "+context_right)  
            # xml entries for the concordancer
            entry_indent = "\n\t"
            parts_indent = "\n\t\t"
            xml_entries.append(entry_indent+"<entry number=\""+str(occur_num)+"\" line=\""+str(line_number+1)+"\">"+parts_indent+"<left>"+context_left+"</left>"+parts_indent+"<middle>"+middle+"</middle>"+parts_indent+"<right>"+context_right+"</right>"+entry_indent+"</entry>")
            occur_num = occur_num+1

In [23]:
for p in plain_conc:
    print(p)
print()
for x in xml_entries:
    print(x)

# Concordancer demo

In [ ]:
from yaml import load, dump
import codecs
import pickle

In [ ]:
unknown = []

In [ ]:
parts = load(codecs.open('syllables_dict.yaml', 'r'))

In [ ]:
with open('syllables_dict.pickle', 'wb') as f:
    pickle.dump(parts, f)

In [ ]:
with open('syllables_dict.pickle', 'rb') as f:
    parts = pickle.load(f)

In [ ]:
text = "གནས་ལུགས་འདི་ལྟར་ལ་སོང་ད་ཆ་ཚོགས་ཆེན་ནས་ཁྱབ་བསྒྲགས་བྱ་རྒྱུར་ཡང་སྙིང་རྗེ་བྲལ་ཞིང་མི་སྤྱོད་ལས་འདས་པ་རབས་ཀྱི་སྤྱོད་ཚུལ་བཅས་ཏེ་མི་མང་གི་འདོད་པ་ནི་གཞུང་གི་དབང་ཆའི་གཞི་རྩར་འཛིན་དགོས་པ་ཡིན་ཞིང"
text = text.split('་')
for num, t in enumerate(text):
    if t in parts.keys():
        if parts[text[num]]['mzh'] == 'པ' and parts[text[num+1]]['y1'] == 'ུ':
            print(text[num]+'་'+text[num+1]+'་''་'+text[num+2])
    else:
        unknown.append(t)

In [ ]:
sa_gc = 0
for t in text:
    if t in parts.keys():
        if parts[t]['gc'] == 'ས':
            sa_gc = sa_gc+1
print(sa_gc)

In [ ]:
unknown

In [ ]:
'ཞིང' in parts.keys()

# Particle agreement

In [24]:
dreldra = []
jedra = []
jungkhung = [["ཏུ", ["ག", "བ"]]]
ladon = [["སུ", ["ས"]], ["ཏུ", ["ག", "བ"]],["རུ", ["འ"]], ["དུ", ["ང", "ད", "ན", "མ", "ར", "ལ"]]]
all_particles = [dreldra, jedra, jungkhung, ladon]

['མ་ཏུ', 'བདག་དུ', 'ཁྲིམས་དུ']


total : 0


# Syllable ordering

In [32]:
def create_sylSignature(syl_dict):
    """
    The function creates the signature of a given syllable.
    The structure is as follows : 
    
    99[000]    [00]   [00]     [00]    [0]    [00]    [00]     [00]    [00]     [00]    [00]
    99[mingzhi][gocan][ngonjug][dogcan][wazur][yang_1][jejug_1][yang_2][jejug_2][yang_3][yangjug]
    
    Each signature starts with 99, in order to recognize the start of a syllable and because the zeros at the beginning of integers aren't taken into account by python.
    Each following position is made of two digits.
    
    The mingzhi's third digit serves to differenciate normal letters (0) and those below a gocan (1).  
    It also serves to put syllables with a gocan after those without.    
    """
    ngj = {"": "00", "ག" : "01", "ད" : "02", "བ" : "03", "མ" : "04", "འ" : "05"}
    gc  = {"" : "00", "ར" : "01", "ལ" : "02", "ས" : "03"}
    mzh = {"ཀ" : "01", "ཁ" : "02", "ག" : "03", "ང" : "04", "ཅ" : "05",  "ཆ" : "06", "ཇ" : "07", "ཉ" : "08", "ཏ" : "09","ཐ" : "10", "ད" : "11", "ན" : "12", "པ" : "13", "ཕ" : "14", "བ" : "15", "མ" : "16", "ཙ" : "17", "ཚ" : "18", "ཛ" : "19", "ཝ" : "20", "ཞ" : "21", "ཟ" : "22", "འ" : "23", "ཡ" : "24", "ར" : "25", "ལ" : "26", "ཤ" : "27", "ས" : "28", "ཧ" : "29", "ཨ" : "30"}
    dc  = {"" : "00", "ཡ" : "01", "ར" : "02", "ལ" : "03"}
    y1  = {"" : "00", "ི" : "01", "ུ" : "02", "ེ" : "03", "ོ" : "04"}
    jj1 = {"" : "00", "ག" : "01", "ང" : "02", "ད" : "03", "ན" : "04", "བ" : "05", "མ" : "06", "འ" : "07", "ར" : "08", "ལ" : "09", "ས" : "10"}
    y2  = y1
    jj2 = jj1
    y3  = y1
    yj  = {"" : "00", "ད" : "01", "ས" : "02"}
    wz  = {"" : "0", "ཝ" : "1"}
    
    pos1 = mzh[syl_dict["mzh"]]
    pos2 = gc[syl_dict["gc"]]
    pos3 = ngj[syl_dict["ngj"]]
    pos4 = dc[syl_dict["dc"]]
    pos5 = wz[syl_dict["wz"]]
    pos6 = y1[syl_dict["y1"]]
    pos7 = jj1[syl_dict["jj1"]]
    pos8 = y2[syl_dict["y2"]]
    pos9 = jj2[syl_dict["jj2"]]
    pos10 = y3[syl_dict["y3"]]
    pos11 = yj[syl_dict["yj"]]

    
    # 3rd mingzhi position : with or without gocan
    if pos2 == "00":
        pos1 = pos1+"0"
    else:
        pos1 = pos1+"1"
    
    return int("99"+pos1+pos2+pos3+pos4+pos5+pos6+pos7+pos8+pos9+pos10+pos11)

In [33]:
def sylFromSignature(signature):
    """
    The function for regenerating the syllable
    from the signature.
    its output is a tuple with the syllable and 
    a dict with its elements.
    """
    ngj = {"00" : "", "01" : "ག", "02" : "ད", "03" : "བ", "04" : "མ", "05" : "འ"}
    gc = {"00" : "", "01" : "ར", "02" : "ལ", "03" : "ས"}
    mzh = {"000" : "", "010" : "ཀ", "011" : "ྐ", "020" : "ཁ", "030" : "ག", "031" : "ྒ", "040" : "ང", "041" : "ྔ", "050" : "ཅ", "051" : "ྕ", "060" : "ཆ", "070" : "ཇ", "071" : "ྗ", "080" : "ཉ", "081" : "ྙ", "090" : "ཏ","091" : "ྟ", "100" : "ཐ", "110" : "ད", "111" : "ྡ", "120" : "ན", "121" : "ྣ", "130" : "པ", "131" : "ྤ", "140" : "ཕ", "150" : "བ", "151" : "ྦ", "160" : "མ", "161" : "ྨ", "170" : "ཙ", "171" : "ྩ", "180" : "ཚ", "190" : "ཛ", "191" : "ྫ", "200" : "ཝ", "210" : "ཞ", "220" : "ཟ", "230" : "འ", "240" : "ཡ", "250" : "ར", "260" : "ལ", "270" : "ཤ", "280" : "ས", "290" : "ཧ", "291" : "ྷ", "300" : "ཨ"}
    dc = {"00" : "", "01" : "ྱ", "02" : "ྲ", "03" : "ླ"}
    y = {"00" : "", "01" : "ི", "02" : "ུ", "03" : "ེ", "04" : "ོ"}
    jj = {"00" : "", "01" : "ག", "02" : "ང", "03" : "ད", "04" : "ན", "05" : "བ", "06" : "མ", "07" : "འ", "08" : "ར", "09" : "ལ", "10" : "ས"}
    yj = {"00" : "", "01" : "ད", "02" : "ས"}
    wz  = {"0" : "", "1" : "ྭ"}

    signature = str(signature)
    pos1 = mzh[signature[2:5]]
    pos2 = gc[signature[5:7]]
    pos3 = ngj[signature[7:9]]
    pos4 = dc[signature[9:11]]
    pos5= wz[signature[23:24]]
    pos6 = y[signature[11:13]]
    pos7 = jj[signature[13:15]]
    pos8 = yj[signature[15:17]]
    pos9 = jj[signature[17:19]]
    pos10 = y[signature[19:21]]
    pos11= yj[signature[21:23]]

    
    norlet = {"": "", "ྐ" : "ཀ", "ྒ" : "ག", "ྔ" : "ང", "ྕ" : "ཅ", "ྗ" : "ཇ", "ྙ" : "ཉ", "ྟ" : "ཏ", "ྡ" : "ད", "ྣ" : "ན", "ྤ" : "པ", "ྦ" : "བ", "ྨ" : "མ", "ྩ" : "ཙ", "ྫ" : "ཛ", "ྭ" : "ཝ", "ྱ" : "ཡ", "ྲ" : "ར", "ླ" : "ལ", "ྷ" : "ཧ"}
    syl_dict = {"ngj" : pos3, "gc" : pos2, "mzh" : norlet[pos1], "dc" : norlet[pos4], "wz" : norlet[pos5], "y1" : pos6, "jj1" : pos7, "y2" : pos8, "jj2" : pos9, "y3" : pos10, "yj" : pos11}
    syllable = pos3+pos2+pos1+pos4+pos5+pos6+pos7+pos8+pos9+pos10+pos11+"་"
    return syllable, syl_dict

In [34]:
currentSyl = {'jj1': '', 'y1': 'ི', 'wz': '', 'ngj': '', 'yj': '', 'y2': '', 'gc': '', 'mzh': 'ཀ', 'jj2': '', 'y3': '', 'dc': ''}
print(create_sylSignature(currentSyl))
currentSyl = {'jj1': '', 'y1': '', 'wz': '', 'ngj': '', 'yj': '', 'y2': '', 'gc': 'ས', 'mzh': 'ཀ', 'jj2': '', 'y3': '', 'dc': ''}
print(create_sylSignature(currentSyl))

99031030302020500010
('བསྒྲུབས་', {'dogcan': 'ར', 'mingzhi': 'ག', 'yang_1': 'ུ', 'jejug': 'བ', 'yangjug': 'ས', 'wazur': '', 'yang_2': '', 'ngonjug': 'བ', 'gocan': 'ས'})


In [ ]:
sig = 990110300000000000000000

print(sylFromSignature(sig))

# Count syllables and letters in a text

## version 0

In [35]:
# python3 syl_letter_count.py file-to-process

In [36]:
import sys
import codecs
import re

In [37]:
#text = codecs.open(sys.argv[1], "r", "utf8").readlines() # line for the command line 
text = codecs.open("./input/a-couple-must-par_11.txt", "r", "utf8").readlines() # line to uncomment in ipython

FileNotFoundError: [Errno 2] No such file or directory: './input/a-couple-must-par_11.txt'

In [38]:
count_syl = {}
count_letter = {}
for line in text:
    line = re.sub("[། {}]", "་", line.strip())
    line = re.sub(r"་+", "་", line)
    syllables = line.split("་")
    for syl in syllables:
        if syl != "":
            if syl in count_syl:
                count_syl[syl] = count_syl[syl]+1
            else:
                count_syl[syl] = 1
            for letter in syl:
                if letter in count_letter:
                    count_letter[letter] = count_letter[letter]+1
                else:
                    count_letter[letter] = 1

NameError: name 'text' is not defined

In [ ]:
syllables = []
letters = []

for syl in count_syl:
    syllables.append((count_syl[syl], syl))
syllables = sorted(syllables, reverse=True)

for letter in count_letter:
    letters.append((count_letter[letter], letter))
letters = sorted(letters, reverse=True)

In [ ]:
sep1 = "_"
sep2 = " "
for syl in syllables:
    print(str(syl[0])+sep1+syl[1], end= sep2)
print("\n")

for l in letters:
    print(str(l[0])+sep1+l[1], end= sep2)
print("\n")

In [ ]:
output1 = codecs.open("a-couple-must-par_11_syl-count.txt", "w", "utf8") # ipython
#output1 = codecs.open(sys.argv[1].split(".")[0]+"_syl-count.txt", "w", "utf8") # command line
for syl in syllables:
    output1.write(str(syl[0])+sep1+syl[1]+"\n")
output1.close()

output2 = codecs.open("a-couple-must-par_11_letter-count.txt", "w", "utf8") # ipython
#output2 = codecs.open(sys.argv[1].split(".")[0]+"_letter-count.txt", "w", "utf8") # command line
for l in letters:
    output2.write(str(l[0])+sep1+l[1]+"\n")
output2.close()

## version 1

In [ ]:
import sys
import codecs
import re
import os

In [ ]:
length = 13 # length of lines
def presentation(liste, unit, per_occ):
    sep = "_"
    out = ""
    total = 0
    for elt in liste:
        total = total + elt[0]
    out += str(total)+unit+"\n"
    counter = 0
    while len(liste)-counter > length:
        for i in range(length):
            occurences = liste[counter][0]
            current = liste[counter][1]
            if per_occ == "percentage":
                percent = "{:.1%}".format(occurences/total)
                out += str(current)+sep+str(percent)+"\t"
            if per_occ == "occurences":
                out += str(current)+sep+str(occurences)+"\t"
            counter = counter +1
        out += "\n"
    for i in range(len(liste)-counter):
        occurences = liste[counter][0]
        current = liste[counter][1]
        if per_occ == "percentage":
            percent = "{:.1%}".format(occurences/total)
            out += str(current)+sep+str(percent)+"\t"
        if per_occ == "occurences":
            out += str(current)+sep+str(occurences)+"\t"
        counter = counter +1
    out += "\n\n"
    return out

In [ ]:
def ordered_list(dictionary):
    """
    takes a dictionary with 
    """
    liste = []
    for entry in dictionary:
        liste.append((dictionary[entry], entry))
    liste = sorted(liste, reverse=True)
    return liste
#print(ordered_list({"a": 3, "b" : 2}))

In [ ]:
all_syl = {}
all_let = {}
os.chdir("./input")
for File in os.listdir("."):
    print(File)
    filename = File.split(".")[0]
    text = codecs.open(File, "r", "utf8").readlines()
    count_syl = {}
    count_letter = {}
    for line in text:
        line = re.sub("[། {}]", "་", line.strip())
        line = re.sub(r"་+", "་", line)
        syllables = line.split("་")
        for syl in syllables:
            if syl != "":
                # counting for each file
                if syl in count_syl:
                    count_syl[syl] = count_syl[syl]+1
                else:
                    count_syl[syl] = 1
                for letter in syl:
                    if letter in count_letter:
                        count_letter[letter] = count_letter[letter]+1
                    else:
                        count_letter[letter] = 1
                # counting for all files
                if syl in all_syl:
                    all_syl[syl] = all_syl[syl]+1
                else:
                    all_syl[syl] = 1
                for letter in syl:
                    if letter in all_let:
                        all_let[letter] = all_let[letter]+1
                    else:
                        all_let[letter] = 1
    
    os.chdir("../output")
    output1 = codecs.open("count_"+filename+".txt", "w", "utf8")
    for elt in ["percentage", "occurences"]:
        output1.write(presentation(ordered_list(count_syl), " syllables", elt))
    for elt in ["percentage", "occurences"]:
        output1.write(presentation(ordered_list(count_letter), " letters", elt))
    output1.close()
    os.chdir("../input")
os.chdir("../")
os.chdir("./output")

output1 = codecs.open("total-count_"+filename+".txt", "w", "utf8")
for elt in ["percentage", "occurences"]:
    output1.write(presentation(ordered_list(all_syl), " syllables", elt))
for elt in ["percentage", "occurences"]:
    output1.write(presentation(ordered_list(all_let), " letters", elt))
output1.close()

os.chdir("../")

## version 2

In [ ]:
import sys
import codecs
import re
import os

In [ ]:
length = 13 # length of lines
def presentation(liste, unit, per_occ):
    sep = "_"
    out = ""
    total = 0
    for elt in liste:
        total = total + elt[0]
    out += str(total)+unit+"\n"
    counter = 0
    while len(liste)-counter > length:
        for i in range(length):
            occurences = liste[counter][0]
            current = liste[counter][1]
            if per_occ == "percentage":
                percent = "{:.1%}".format(occurences/total)
                out += str(current)+sep+str(percent)+"\t"
            if per_occ == "occurences":
                out += str(current)+sep+str(occurences)+"\t"
            counter = counter +1
        out += "\n"
    for i in range(len(liste)-counter):
        occurences = liste[counter][0]
        current = liste[counter][1]
        if per_occ == "percentage":
            percent = "{:.1%}".format(occurences/total)
            out += str(current)+sep+str(percent)+"\t"
        if per_occ == "occurences":
            out += str(current)+sep+str(occurences)+"\t"
        counter = counter +1
    out += "\n\n"
    return out

In [ ]:
def len_list_sort(super_list):
    temp = []
    out = []
    for liste in super_list:
        temp.append((len(liste), liste))
    temp = sorted(temp, reverse=True)
    for t in temp:
        out.append(t[1])
    return len(out[0])

In [ ]:
def csv_presentation(tuple_list):
    total = 0
    for elt in tuple_list:
        total = total + elt[0]
    entries = ["Total Syllables"]
    occurences = [str(total)]
    percentages = ["100%"]
    for t in tuple_list:
        entries.append(t[1]+"་")
        occurences.append(str(t[0]))
        percentages.append("{:.1%}".format(t[0]/total))
    return [entries,percentages, occurences]
    #print(entries, occurences, percentages)

In [ ]:
def csv_output(super_list):
    """
    takes a list of lists and outputs a string where each column is 
    one on the second-level list.
    requires len_list_sort()
    """
    out = ""
    longrow = len_list_sort(super_list)
    cols = len(super_list)
    for l in range(longrow):
        for c in range(cols):
            if l < len(super_list[c]):
                out += super_list[c][l]+"\t"
            else:
                out += "\t"
        out += "\n"
    return out
#print(csv_output([["1","5","9","13"],["2","6","10"],["3","7","11","14","15","16","23"],["4","8"]]))

In [ ]:
all_syl = {}
all_let = {}

In [ ]:
os.chdir("./input")
for File in os.listdir("."):
    print(File)
    filename = File.split(".")[0]
    text = codecs.open(File, "r", "utf8").readlines()
    count_syl = {}
    count_letter = {}
    for line in text:
        line = re.sub("[། {}]", "་", line.strip())
        line = re.sub(r"་+", "་", line)
        syllables = line.split("་")
        for syl in syllables:
            if syl != "":
                # counting for each file
                if syl in count_syl:
                    count_syl[syl] = count_syl[syl]+1
                else:
                    count_syl[syl] = 1
                for letter in syl:
                    if letter in count_letter:
                        count_letter[letter] = count_letter[letter]+1
                    else:
                        count_letter[letter] = 1
                # counting for all files
                if syl in all_syl:
                    all_syl[syl] = all_syl[syl]+1
                else:
                    all_syl[syl] = 1
                for letter in syl:
                    if letter in all_let:
                        all_let[letter] = all_let[letter]+1
                    else:
                        all_let[letter] = 1
    
    os.chdir("../output")
    output1 = codecs.open("count_"+filename+".txt", "w", "utf8")
    for elt in ["percentage", "occurences"]:
        output1.write(presentation(ordered_list(count_syl), " syllables", elt))
    for elt in ["percentage", "occurences"]:
        output1.write(presentation(ordered_list(count_letter), " letters", elt))
    output1.close()
    os.chdir("../input")
os.chdir("../")
os.chdir("./output")

output2 = codecs.open("total-count_"+filename+".csv", "w", "utf8")
output2.write(csv_output(csv_presentation(ordered_list(all_syl))+csv_presentation(ordered_list(all_let))))
output2.close()

os.chdir("../")

# TODO : qqch qui détecte les malformations à partir de tsetan shabdrung

In [39]:
# No stack
salje30 = ["ཀ", "ཁ", "ག", "ང", "ཅ", "ཆ", "ཇ", "ཉ", "ཏ", "ཐ", "ད", "ན", "པ", "ཕ", "བ", "མ", "ཙ", "ཚ", "ཛ", "ཝ", "ཞ", "ཟ", "འ", "ཡ", "ར", "ལ", "ཤ", "ས", "ཧ", "ཨ"]

# 2 letters stack
raGocan12 = ["རྐ", "རྒ", "རྔ", "རྗ", "རྙ", "རྟ", "རྡ", "རྣ", "རྦ", "རྨ", "རྩ", "རྫ"]
laGocan10 = ["ལྐ", "ལྒ", "ལྔ", "ལྕ", "ལྗ", "ལྟ", "ལྡ", "ལྤ", "ལྦ", "ལྷ"]
saGocan11 = ["སྐ", "སྒ", "སྔ", "སྙ", "སྟ", "སྡ", "སྣ", "སྤ", "སྦ", "སྨ", "སྩ"]

yaDogcan9 = ["ཀྱ", "ཁྱ", "གྱ", "པྱ", "ཕྱ", "བྱ", "མྱ", "དྱ", "ཧྱ"] # ཡ་འདོགས་ཅན་བདུན་ལ་མི་འགལ་བ་གཉིས་མནན་པས་དགུ་ཡིན།
raDogcan12 = ["ཀྲ", "ཁྲ", "གྲ", "ཏྲ", "ཐྲ","དྲ", "པྲ", "ཕྲ", "བྲ", "མྲ", "སྲ", "ཧྲ"]
laDogcan6 = ["ཀླ", "གླ", "བླ", "ཟླ", "རླ", "སླ"]

# 3 letters stack
yatagSumtseg8 = ["རྐྱ", "རྒྱ", "རྨྱ", "སྐྱ", "སྒྱ", "སྤྱ", "སྦྱ", "སྨྱ"]
ratagSumtseg6 = ["སྐྲ", "སྒྲ", "སྣྲ", "སྤྲ", "སྦྲ", "སྨྲ"]

In [40]:
gocan33 =  raGocan12 + laGocan10 + saGocan11
dogcan27 = yaDogcan9 + raDogcan12 + laDogcan6 # འདོགས་ཅན་ཉེར་ལྔ་ལ་ཡ་འདོགས་ཅན་དམིགས་གསལ་གཉིས་མནན་པས་ཉེར་བདུན་ཡིན།
sumtseg14 = yatagSumtseg8 + ratagSumtseg6 
tsegYig74 = gocan33 + dogcan27 + sumtseg14 # གོང་གསལ་གཞིན་དོན་གཉིས་ལ་གཉིས་མནན་པས་དོན་བཞི་ཡིན།

# Grand total
kyangTsegDogKun = salje30 + tsegYig74

In [41]:
salje_parts = {"ཀ" : {"gocan" : "", "mingzhi" : "ཀ", "dogcan" : ""}, "ཁ" : {"gocan" : "", "mingzhi" : "ཁ", "dogcan" : ""}, "ག" : {"gocan" : "", "mingzhi" : "ག", "dogcan" : ""}, "ང" : {"gocan" : "", "mingzhi" : "ང", "dogcan" : ""}, "ཅ" : {"gocan" : "", "mingzhi" : "ཅ", "dogcan" : ""}, "ཆ" : {"gocan" : "", "mingzhi" : "ཆ", "dogcan" : ""}, "ཇ" : {"gocan" : "", "mingzhi" : "ཇ", "dogcan" : ""}, "ཉ" : {"gocan" : "", "mingzhi" : "ཉ", "dogcan" : ""}, "ཏ" : {"gocan" : "", "mingzhi" : "ཏ", "dogcan" : ""}, "ཐ" : {"gocan" : "", "mingzhi" : "ཐ", "dogcan" : ""}, "ད" : {"gocan" : "", "mingzhi" : "ད", "dogcan" : ""}, "ན" : {"gocan" : "", "mingzhi" : "ན", "dogcan" : ""}, "པ" : {"gocan" : "", "mingzhi" : "པ", "dogcan" : ""}, "ཕ" : {"gocan" : "", "mingzhi" : "ཕ", "dogcan" : ""}, "བ" : {"gocan" : "", "mingzhi" : "བ", "dogcan" : ""}, "མ" : {"gocan" : "", "mingzhi" : "མ", "dogcan" : ""}, "ཙ" : {"gocan" : "", "mingzhi" : "ཙ", "dogcan" : ""}, "ཚ" : {"gocan" : "", "mingzhi" : "ཚ", "dogcan" : ""}, "ཛ" : {"gocan" : "", "mingzhi" : "ཛ", "dogcan" : ""}, "ཝ" : {"gocan" : "", "mingzhi" : "ཝ", "dogcan" : ""}, "ཞ" : {"gocan" : "", "mingzhi" : "ཞ", "dogcan" : ""}, "ཟ" : {"gocan" : "", "mingzhi" : "ཟ", "dogcan" : ""}, "འ" : {"gocan" : "", "mingzhi" : "འ", "dogcan" : ""}, "ཡ" : {"gocan" : "", "mingzhi" : "ཡ", "dogcan" : ""}, "ར" : {"gocan" : "", "mingzhi" : "ར", "dogcan" : ""}, "ལ" : {"gocan" : "", "mingzhi" : "ལ", "dogcan" : ""}, "ཤ" : {"gocan" : "", "mingzhi" : "ཤ", "dogcan" : ""}, "ས" : {"gocan" : "", "mingzhi" : "ས", "dogcan" : ""}, "ཧ" : {"gocan" : "", "mingzhi" : "ཧ", "dogcan" : ""}, "ཨ" : {"gocan" : "", "mingzhi" : "ཨ", "dogcan" : ""}}
ragocan_parts = {"རྐ" : {"gocan" : "ར", "mingzhi" : "ཀ", "dogcan" : ""}, "རྒ" : {"gocan" : "ར", "mingzhi" : "ག", "dogcan" : ""}, "རྔ" : {"gocan" : "ར", "mingzhi" : "ང", "dogcan" : ""}, "རྗ" : {"gocan" : "ར", "mingzhi" : "ཇ", "dogcan" : ""}, "རྙ" : {"gocan" : "ར", "mingzhi" : "ཉ", "dogcan" : ""}, "རྟ" : {"gocan" : "ར", "mingzhi" : "ཏ", "dogcan" : ""}, "རྡ" : {"gocan" : "ར", "mingzhi" : "ད", "dogcan" : ""}, "རྣ" : {"gocan" : "ར", "mingzhi" : "ན", "dogcan" : ""}, "རྦ" : {"gocan" : "ར", "mingzhi" : "བ", "dogcan" : ""}, "རྨ" : {"gocan" : "ར", "mingzhi" : "མ", "dogcan" : ""}, "རྩ" : {"gocan" : "ར", "mingzhi" : "ཙ", "dogcan" : ""}, "རྫ" : {"gocan" : "ར", "mingzhi" : "ཛ", "dogcan" : ""}}
lagocan_parts = {"ལྐ" : {"gocan" : "ལ", "mingzhi" : "ཀ", "dogcan" : ""}, "ལྒ" : {"gocan" : "ལ", "mingzhi" : "ག", "dogcan" : ""}, "ལྔ" : {"gocan" : "ལ", "mingzhi" : "ང", "dogcan" : ""}, "ལྕ" : {"gocan" : "ལ", "mingzhi" : "ཅ", "dogcan" : ""}, "ལྗ" : {"gocan" : "ལ", "mingzhi" : "ཇ", "dogcan" : ""}, "ལྟ" : {"gocan" : "ལ", "mingzhi" : "ཏ", "dogcan" : ""}, "ལྡ" : {"gocan" : "ལ", "mingzhi" : "ད", "dogcan" : ""}, "ལྤ" : {"gocan" : "ལ", "mingzhi" : "པ", "dogcan" : ""}, "ལྦ" : {"gocan" : "ལ", "mingzhi" : "བ", "dogcan" : ""}, "ལྷ" : {"gocan" : "ལ", "mingzhi" : "ཧ", "dogcan" : ""}}
sagocan_parts = {"སྐ" : {"gocan" : "ས", "mingzhi" : "ཀ", "dogcan" : ""}, "སྒ" : {"gocan" : "ས", "mingzhi" : "ག", "dogcan" : ""}, "སྔ" : {"gocan" : "ས", "mingzhi" : "ང", "dogcan" : ""}, "སྙ" : {"gocan" : "ས", "mingzhi" : "ཉ", "dogcan" : ""}, "སྟ" : {"gocan" : "ས", "mingzhi" : "ཏ", "dogcan" : ""}, "སྡ" : {"gocan" : "ས", "mingzhi" : "ད", "dogcan" : ""}, "སྣ" : {"gocan" : "ས", "mingzhi" : "ན", "dogcan" : ""}, "སྤ" : {"gocan" : "ས", "mingzhi" : "པ", "dogcan" : ""}, "སྦ" : {"gocan" : "ས", "mingzhi" : "བ", "dogcan" : ""}, "སྨ" : {"gocan" : "ས", "mingzhi" : "མ", "dogcan" : ""}, "སྩ" : {"gocan" : "ས", "mingzhi" : "ཙ", "dogcan" : ""}}
yadogcan_parts = {"ཀྱ" : {"gocan" : "","mingzhi" : "ཀ", "dogcan" : "ཡ"}, "ཁྱ" : {"gocan" : "","mingzhi" : "ཁ", "dogcan" : "ཡ"}, "གྱ" : {"gocan" : "","mingzhi" : "ག", "dogcan" : "ཡ"}, "པྱ" : {"gocan" : "","mingzhi" : "པ", "dogcan" : "ཡ"}, "ཕྱ" : {"gocan" : "","mingzhi" : "ཕ", "dogcan" : "ཡ"}, "བྱ" : {"gocan" : "","mingzhi" : "བ", "dogcan" : "ཡ"}, "མྱ" : {"gocan" : "","mingzhi" : "མ", "dogcan" : "ཡ"}, "དྱ" : {"gocan" : "","mingzhi" : "ད", "dogcan" : "ཡ"}, "ཧྱ" : {"gocan" : "","mingzhi" : "ཧ", "dogcan" : "ཡ"}}
radogcan_parts = {"gocan" : "","ཀྲ" : {"gocan" : "", "mingzhi" : "ཀ", "dogcan" : "ར"}, "ཁྲ" : {"gocan" : "","mingzhi" : "ཁ", "dogcan" : "ར"}, "གྲ" : {"gocan" : "","mingzhi" : "ག", "dogcan" : "ར"}, "ཏྲ" : {"gocan" : "","mingzhi" : "ཏ", "dogcan" : "ར"}, "ཐྲ" : {"gocan" : "","mingzhi" : "ཐ", "dogcan" : "ར"}, "དྲ" : {"gocan" : "","mingzhi" : "ད", "dogcan" : "ར"}, "པྲ" : {"gocan" : "","mingzhi" : "པ", "dogcan" : "ར"}, "ཕྲ" : {"gocan" : "","mingzhi" : "ཕ", "dogcan" : "ར"}, "བྲ" : {"gocan" : "","mingzhi" : "བ", "dogcan" : "ར"}, "མྲ" : {"gocan" : "","mingzhi" : "མ", "dogcan" : "ར"}, "སྲ" : {"gocan" : "","mingzhi" : "ས", "dogcan" : "ར"}, "ཧྲ" : {"gocan" : "","mingzhi" : "ཧ", "dogcan" : "ར"}}
ladogcan_parts = {"ཀླ" : {"gocan" : "","mingzhi" : "ཀ", "dogcan" : "ལ"}, "གླ" : {"gocan" : "","mingzhi" : "ག", "dogcan" : "ལ"}, "བླ" : {"gocan" : "","mingzhi" : "བ", "dogcan" : "ལ"}, "ཟླ" : {"gocan" : "","mingzhi" : "ཟ", "dogcan" : "ལ"}, "རླ" : {"gocan" : "","mingzhi" : "ར", "dogcan" : "ལ"}, "སླ" : {"gocan" : "","mingzhi" : "ས", "dogcan" : "ལ"}}
yatagSumtseg_parts = {"རྐྱ" : {"gocan" : "ར", "mingzhi" : "ཀ", "dogcan" : "ཡ"}, "རྒྱ" : {"gocan" : "ར", "mingzhi" : "ག", "dogcan" : "ཡ"}, "རྨྱ" : {"gocan" : "ར", "mingzhi" : "མ", "dogcan" : "ཡ"}, "སྐྱ" : {"gocan" : "ས", "mingzhi" : "ཀ", "dogcan" : "ཡ"}, "སྒྱ" : {"gocan" : "ས", "mingzhi" : "ག", "dogcan" : "ཡ"}, "སྤྱ" : {"gocan" : "ས", "mingzhi" : "པ", "dogcan" : "ཡ"}, "སྦྱ" : {"gocan" : "ས", "mingzhi" : "བ", "dogcan" : "ཡ"}, "སྨྱ" : {"gocan" : "ས", "mingzhi" : "མ", "dogcan" : "ཡ"}}
ratagSumtseg_parts = {"སྐྲ" : {"gocan" : "ས", "mingzhi" : "ཀ", "dogcan" : "ར"}, "སྒྲ" : {"gocan" : "ས", "mingzhi" : "ག", "dogcan" : "ར"}, "སྣྲ" : {"gocan" : "ས", "mingzhi" : "ན", "dogcan" : "ར"}, "སྤྲ" : {"gocan" : "ས", "mingzhi" : "པ", "dogcan" : "ར"}, "སྦྲ" : {"gocan" : "ས", "mingzhi" : "བ", "dogcan" : "ར"}, "སྨྲ" : {"gocan" : "ས", "mingzhi" : "མ", "dogcan" : "ར"}}
all_parts_dict = {}
for l in [salje_parts, ragocan_parts, lagocan_parts, sagocan_parts, yadogcan_parts, radogcan_parts, ladogcan_parts, yatagSumtseg_parts,ratagSumtseg_parts ]:
    all_parts_dict.update(l)

In [42]:
gePhulKyangpa11 = ["ཅ", "ཉ", "ཏ", "ད", "ན", "ཙ", "ཞ", "ཟ", "ཡ", "ཤ", "ས"]

dePhulKyangpa6 = ["ཀ", "ག", "ང", "པ", "བ", "མ"]
dePhulDogcan10 = ["ཀྱ", "ཀྲ", "གྱ", "གྲ", "པྱ", "པྲ", "བྱ", "བྲ", "མྱ", "མྲ"]

bePhulKyangpa10 = ["ཀ", "ག", "ཅ", "ཏ", "ད", "ཙ", "ཞ", "ཟ", "ཤ", "ས"]
bePhulDogcan7 = ["ཀྱ", "ཀྲ", "ཀླ", "གྱ", "ཟླ", "གླ", "གྲ"]
bePhulRaGocan11 = ["རྐ", "རྒ", "རྔ", "རྗ", "རྙ", "རྟ", "རྔ", "རྣ", "རྩ", "རྫ", "རླ"]
bePhulLaGocan2 = ["ལྟ", "ལྡ"]
bePhulSaGocan10 = ["སྐ", "སྒ", "སྔ", "སྙ", "སྟ", "སྡ", "སྣ", "སྩ", "སྲ", "སླ"]
bePhulSumtseg6 = ["རྐྱ", "རྒྱ", "སྐྱ", "སྐྲ", "སྒྱ", "སྒྲ"]

mePhulKyangpa11 = ["ཁ", "ག", "ང", "ཆ", "ཇ", "ཉ", "ཐ", "ད", "ན", "ཚ", "ཛ"]
mePhulDogcan4 = ["ཁྱ", "ཁྲ", "གྱ", "གྲ"]

ePhulKyangpa10 = ["ཁ", "ག", "ཆ", "ཇ", "ཐ", "ད", "ཕ", "བ", "ཚ", "ཛ"]
ePhulDogcan9 = ["ཁྱ", "ཁྲ", "གྱ", "གྲ", "དྲ", "ཕྲ", "ཕྱ", "བྱ", "བྲ"]

gePhul11 = gePhulKyangpa11
bePhul48 = bePhulKyangpa10 + bePhulDogcan7 + bePhulRaGocan11 + bePhulLaGocan2 + bePhulSaGocan10 + bePhulSumtseg6
dePhul16 = dePhulKyangpa6 + dePhulDogcan10
mePhul15 = mePhulKyangpa11 + mePhulDogcan4
ePhul19 = ePhulKyangpa10 + ePhulDogcan9

phulyig107 = gePhul11 + bePhul48 + dePhul16 + mePhul15 + ePhul19

In [43]:
jejug10 = ["", "ག", "ང", "ད", "ན", "བ", "མ", "འ", "ར", "ལ", "ས"]
ngonjug5 = ["", "ག", "ད", "བ", "མ", "འ"]
yang4 = ["", "ི", "ུ", "ེ", "ོ"] 
yangjug1 = ["", "ས"]
yangjugcan4 = ["ག", "ང", "བ", "མ"]

In [44]:
syllables = []

# [mingzhi+gocan+dogcan] > ngonjug > yang > jejug > yangjug > wazur
for stack in kyangTsegDogKun:
    ngonjug = ""
    gocan = ""
    mingzhi = ""
    dogcan = ""
    yang = ""
    jejug = ""
    yangjug = ""
    
    # 1. attribute values to the main stack of letter
    gocan = all_parts_dict[stack]["gocan"]
    mingzhi = all_parts_dict[stack]["mingzhi"]
    dogcan = all_parts_dict[stack]["dogcan"]
        
    for n in ngonjug5:
        # 2. attribute the right value to the prefix
        if n == "ག" and stack in gePhul11:
            ngonjug = n
        elif n == "ད" and stack in dePhul16:
            ngonjug = n
        elif n == "བ" and stack in bePhul48:
            ngonjug = n
        elif n == "མ" and stack in mePhul15:
            ngonjug = n
        elif n == "འ" and stack in ePhul19:
            ngonjug = n
            
        for y in yang4:
            # 3. attribute a value to the vowel
            yang = y
            
            for j in jejug10:
                # 4. attribute a value to the suffix
                jejug = j
                
                for ya in yangjug1:
                    # 5. attribute a value to the post-suffix or not
                    if jejug in yangjugcan4:
                        yangjug = ya
                    
                    # 6. adding a achung to cases where there is a prefix, 
                    #    a main letter but no suffix.
                    # །རྐྱང་པ་འཕུལ་བ་འ་ཡི་མཐའ། །དགོས་ཀྱི་གུག་སྐྱེད་ཅན་ལ་མིན།
                    if jejug == "" and mingzhi != "" and ngonjug != "" and yang == "" and gocan == "" and dogcan == "":
                        jejug = "འ"
                    
                    # 7. removing unnecessary འ
                    if jejug == "འ":
                        if ngonjug == "" and gocan == "" and dogcan == "" and yang == "":
                            jejug = ""
                        elif yang != "":
                            jejug = ""
                        elif gocan != "" or dogcan != "":
                            jejug = ""
                        elif gocan != "" and dogcan != "":
                            jejug = ""
                    
                    syllable = ngonjug+ stack  +yang+jejug+yangjug
                    parts = {"ngonjug": ngonjug, "gocan": gocan, "mingzhi": mingzhi, "dogcan": dogcan, "wazur": "", "yang_1": yang, "jejug": jejug, "yang_2": "", "yangjug": yangjug}
                    if syllable not in syllables :
                        syllables.append((syllable, parts))
                    yangjug = ""
                jejug = ""
            yang = ""

# adding exceptions ["མདྲོན", "བགླ", "མདྲ", "དྭགས", "དྭངས", "དྭབས", "དྭམས", "མྭགས", "མྭངས", "འྭགས", "འྭབས"]
migsel = [("མདྲོན" , {"ngonjug": "མ", "gocan": "", "mingzhi": "ད", "dogcan": "ར", "wazur": "", "yang_1": "ོ", "jejug": "ན", "yang_2": "", "yangjug": ""}), ("བགླ" , {"ngonjug": "བ", "gocan": "", "mingzhi": "ག", "dogcan": "ལ", "wazur": "", "yang_1": "", "jejug": "", "yang_2": "", "yangjug": ""}), ("མདྲ" , {"ngonjug": "མ", "gocan": "", "mingzhi": "ད", "dogcan": "ར", "wazur": "", "yang_1": "", "jejug": "", "yang_2": "", "yangjug": ""}), ("དྭགས" , {"ngonjug": "", "gocan": "", "mingzhi": "ད", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "ག", "yang_2": "", "yangjug": "ས"}), ("དྭངས" , {"ngonjug": "", "gocan": "", "mingzhi": "ད", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "ང", "yang_2": "", "yangjug": "ས"}), ("དྭབས" , {"ngonjug": "", "gocan": "", "mingzhi": "ད", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "བ", "yang_2": "", "yangjug": "ས"}), ("དྭམས" , {"ngonjug": "", "gocan": "", "mingzhi": "ད", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "མ", "yang_2": "", "yangjug": "ས"}), ("མྭགས" , {"ngonjug": "", "gocan": "", "mingzhi": "མ", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "ག", "yang_2": "", "yangjug": "ས"}), ("མྭངས" , {"ngonjug": "", "gocan": "", "mingzhi": "མ", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "ང", "yang_2": "", "yangjug": "ས"}), ("འྭགས" , {"ngonjug": "", "gocan": "", "mingzhi": "འ", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "ག", "yang_2": "", "yangjug": "ས"}), ("འྭབས" , {"ngonjug": "", "gocan": "", "mingzhi": "འ", "dogcan": "", "wazur": "ཝ", "yang_1": "", "jejug": "བ", "yang_2": "", "yangjug": "ས"})]
for m in migsel:
    syllables.append(m)

In [45]:
syllables_singled = []
for syl in syllables:
    if syl not in syllables_singled:
        syllables_singled.append(syl)
print(len(syllables_singled))

KeyboardInterrupt: 

In [ ]:
count = 0
for nb, s in enumerate(syllables_singled):
    if s[1]["jejug"] == "འ" or s[1]["jejug"] == "":
        #print(s)
        print(s[0], end= "་ ")
        count = count + 1
print()
print(count)

In [ ]:
new_list = []
#test= [('ཀི་', {'ngonjug': '', 'yang_2': '', 'jejug': '', 'mingzhi': 'ཀ', 'yangjug': '', 'gocan': '', 'dogcan': '', 'yang_1': 'ི', 'wazur': ''}), ('ཀོའ་', {'ngonjug': '', 'yang_2': '', 'jejug': 'འ', 'mingzhi': 'ཀ', 'yangjug': '', 'gocan': '', 'dogcan': '', 'yang_1': 'ོ', 'wazur': ''}), ('ཀུ་', {'ngonjug': '', 'yang_2': '', 'jejug': '', 'mingzhi': 'ཀ', 'yangjug': '', 'gocan': '', 'dogcan': '', 'yang_1': 'ུ', 'wazur': ''})]
for s in syllables_singled:
    if s[1]["jejug"] == "འ":
        for y in ["ི", "ོ", "ུ"]:
            parts = s[1]
            parts["yang_2"] = y
            syllable = s[0].split("་")[0]+y+"་"
            new_list.append((syllable, parts))
    if s[1]["jejug"] == "":
        for y in ["ི", "ོ", "ུ"]:
            parts = s[1]
            parts["jejug"] = "འ"
            parts["yang_2"] = y 
            syllable = s[0].split("་")[0]+"འ"+y+"་"
            new_list.append((syllable, parts))

for n in new_list:
    print(n[0], end=" ")
new_list[0]

In [ ]:
#searching for a syllable from a character included in it.

count = 0
for nb, s in enumerate(syllables_singled):
    if "མ" in s[0]: 
        print(s[0], end = "་ ")
        count = count + 1
print()
print(count)